In [ ]:
from data_manager import DataManager

imdb_data = DataManager(
    dbname='imdb',
    user='samaher',
    password="CodingIsFun++",
    host='localhost',
    port='5432'
)

# Get the movie data from the database
movie_data = imdb_data.get_movie()

In [ ]:
movie_data

In [ ]:
import numpy as np
import pandas as pd
import random 


In [ ]:
def find_highest_voted_movie(movie_df):
    max_votes_index = np.argmax(movie_df['votes'].values)
    highest_voted_movie = movie_df.loc[max_votes_index, 'official_title']
    return highest_voted_movie

In [ ]:
# Test the function with the provided movie_data
highest_voted = find_highest_voted_movie(movie_data)
print("The movie with the highest number of votes is:", highest_voted)

In [ ]:
def generate_question(row, desired_difficulty, question_type, correct_answer_column):
    official_title = row['official_title']
    correct_answer = row[correct_answer_column]

    # Determining difficulty level 
    year = int(row['year']) 
    votes = int(row['votes'])
    difficulty_level = determine_difficulty_level(year, votes)

    # Checking if the difficulty level matches the desired difficulty
    if difficulty_level == desired_difficulty:
        
        # Getting unique values excluding the correct answer
        other_options = list(set(movie_data[correct_answer_column].unique()))
        other_options.remove(correct_answer)  # Removing the correct answer

        # Selecting 3 more random incorrect options
        all_options = [correct_answer] + random.sample(other_options, 3)

        # Shuffling the options to randomize their order
        random.shuffle(all_options)

        # Mapping options to letters (A, B, C, D)
        options_mapping = {chr(ord('A') + i): option for i, option in enumerate(all_options)}

        # Constructing the question dictionary
        question_dict = {
            'question': f'{question_type} || {official_title} ||',
            'options': options_mapping,
            'correct_answer': chr(ord('A') + other_options.index(correct_answer)),
            'difficulty_level': difficulty_level
        }

        return question_dict


In [ ]:
def quiz_game(data_frame=None):
    if data_frame is None:
        # If no dataframe is provided, we'll movie_data as default
        data_frame = movie_data

    difficulty_levels = ['easy', 'medium', 'hard']
    total_score = 0
    used_questions = []

    # Getting user input for difficulty level
    user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # List of question generators with parameters for different question types
    question_generators = [
        {'generator': generate_question, 'params': ('When was this movie released? ==> ', 'year')},
        {'generator': generate_question, 'params': ('Where was this movie produced? ==> ', 'country')}
    ]

    # Playing 10 questions, randomly choosing from different types of questions
    for i in range(10):
        while True:
            # Randomly select a question generator with its respective parameters
            question_info = None
            generator_info = random.choice(question_generators)
            question_type, correct_answer_column = generator_info['params']

            row_index = random.randint(0, len(data_frame) - 1)
            row = data_frame.iloc[row_index]

            # Generating the question with the desired difficulty level using the selected generator
            question_info = generator_info['generator'](row, user_difficulty, question_type, correct_answer_column)

            # Making sure used questions do not appear again in the same quiz
            if question_info is not None and question_info['question'] not in used_questions:
                used_questions.append(question_info['question'])
                break

        # Printing the question information
        print(question_info['question'])

        # Printing options with letters (A, B, C, D)
        for letter, option in question_info['options'].items():
            print(f"{letter}. {option}")

        # Getting user's choice with validation
        user_choice = None
        while user_choice not in ['A', 'B', 'C', 'D']:
            user_choice = input("Entering your choice (A, B, C, D): ").upper()

            if user_choice not in ['A', 'B', 'C', 'D']:
                print("You typed a wrong letter. Please type again.")

        # Checking if the user's choice is correct
        is_correct = user_choice == question_info['correct_answer']

        # Calculating and displaying the score
        score = calculate_score(question_info['difficulty_level'], is_correct)
        total_score += score

        # Providing feedback on the answer
        if is_correct:
            print("Correct!")
        else:
            print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

        print(f"Your score for this question: {score}")
        print("----------------------------")

    print(f"Total score: {total_score}")


In [ ]:
imdb_data.close_connection()

In [ ]:
quiz_game()